# Harmonised API performance benchmarking

Tests for Jupyter Hub performance:
    1. Download speed
    2. Data allowances
    3. Processing capability

In [ ]:
import os
import sys
import json
import time
from zipfile import ZipFile
sys.path.append(os.path.dirname(os.getcwd()) + '/Hub_Tools/')
import harmonised_data_access_api_tools as hapi

In [ ]:
api_key = "YOUR API KEY"
download_dir_path = "/home/jovyan/work/products"
JSON_query_dir = os.path.join(os.getcwd(),'JSON_templates')
verbose = False

In [ ]:
# example data sets available: codes from here >> https://www.wekeo.eu/dataset-navigator/start
dataset_ids = []
# ---------------------------- CMEMS test options --------------------------
#dataset_ids.append("EO:MO:DAT:OCEANCOLOUR_GLO_CHL_L4_REP_OBSERVATIONS_009_093") # ESA OC-CCI REP
# ---------------------------- C3S test options ----------------------------
#dataset_ids.append("EO:ECMWF:DAT:ERA5_HOURLY_DATA_ON_SINGLE_LEVELS_1979_PRESENT") # ERA5
# ---------------------------- CAMS test options ---------------------------
#dataset_ids.append("EO:ECMWF:DAT:CAMS_SOLAR_RADIATION_TIMESERIES") # SOLAR RAD
# ----------------- EUMETSAT COPERNICUS MARINE test options ----------------
#dataset_ids.append("EO:EUM:DAT:SENTINEL-3:OL_1_EFR___") # OLCI EFR
# ------------------------- ESA MARINE options test options ----------------
#dataset_ids.append("EO:ESA:DAT:SENTINEL-2:MSI1C")       # MSI L1C
dataset_ids.append("EO:ESA:DAT:SENTINEL-1:L1_GRD")      # S1 L1 GRD

In [ ]:
# find query file
JSON_query_files = []
for dataset_id in dataset_ids:
    JSON_query_file = os.path.join(JSON_query_dir,'Download_benchmarking',dataset_id.replace(':','_')+".json")
    if not os.path.exists(JSON_query_file):
        print('Query file ' + JSON_query_file + ' does not exist')
    else:
        print('Found JSON query file for '+dataset_id)
    JSON_query_files.append(JSON_query_file)

In [ ]:
#DOWNLOAD TEST
for ii in range(5):
    for dataset_id,JSON_query_file in zip(dataset_ids, JSON_query_files):

        print(dataset_id)

        HAPI_dict = hapi.init(dataset_id, api_key, download_dir_path, verbose=verbose)
        HAPI_dict = hapi.get_access_token(HAPI_dict)
        HAPI_dict = hapi.accept_TandC(HAPI_dict)

        # load the query
        with open(JSON_query_file, 'r') as f:
            query = json.load(f)

        # launch job
        HAPI_dict = hapi.launch_query(HAPI_dict, query)

        # wait for jobs to complete
        hapi.check_job_status(HAPI_dict)

        # check results
        HAPI_dict = hapi.get_results_list(HAPI_dict)
        HAPI_dict = hapi.get_download_links(HAPI_dict)

        t0 = time.time()
        # download data
        HAPI_dict = hapi.download_data(HAPI_dict, skip_existing=False)
        
        # flush benchmark timing to stdout print
        print(' ')
        print('--------------------------------')
        print('Elapsed time: ' +str(time.time() - t0))
        print('--------------------------------')
        print('\r')
        